In [2]:
import pandas as pd
import json

In [3]:
df = pd.read_csv("../Dataset/dataset.csv")
df.head()

,author,data,lyrics,style,title
0,Midian Lima,NaN,"Amanheceu,Nada pesquei,Parecia ser apenas mais...",eletrônica,Não Pare
1,MC Doni,NaN,"Não adianta tu se declarar,Romance, compromiss...",eletrônica,Te Amo Sem Compromisso (To Nem Aí)
2,Sebastián Yatra,NaN,"I remember when I met you,I didn't wanna fall,...",eletrônica,My Only One (No Hay Nadie Más) (part. Isabela ...
3,Billie Eilish,NaN,"Thought I found a way,Thought I found a way ou...",eletrônica,Lovely (feat. Khalid)
4,Ed Sheeran,NaN,"I found a love for me,Darling, just dive right...",eletrônica,Perfect


In [4]:
from nltk import word_tokenize
import string
def clusterfy(sentence, clt):
    #cleaning the sentence
    if not isinstance(sentence, str):
        return []
    for p in string.punctuation:
        sentence = sentence.replace(p, " ")
    sentence = sentence.replace("  ", " ").lower()
    
    
    tokens = word_tokenize(sentence)
    for i,token in enumerate(tokens):
            if token in clt["clusters"]:
                cluster_id = clt["clusters"][token]
                tokens[i] = clt["replace"][str(cluster_id)]
                
    return tokens

## Creating the bag of words with the k-means clusters

In [2]:
with open("k_means_clusters.json", "r") as f:
    kmeans_clusters = json.load(f)

In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import  RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier

In [6]:
from functools import reduce
import operator

_id = reduce(
    operator.add,
    [
        df["style"] == "pop",
        df["style"] == "rock",
        df["style"] == "reggae",
        df["style"] == "indie",
        df["style"] == "country"
    ]
)
df = df.loc[_id]
df.head()

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/pandas/core/computation/expressions.py:194: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  op=op_str, alt_op=unsupported[op_str]


,author,data,lyrics,style,title
73,Michael Jackson,NaN,"Hee! Hee!,Ooh!,Go on girl!,Aaow!,Hey, pretty b...",pop,The Way You Make Me Feel
74,Backstreet Boys,NaN,"Hmm, yeah, yeah,Baby, please try to forgive me...",pop,Shape Of My Heart
75,Rihanna,NaN,"I've been everywhere, man,Looking for someone,...",pop,Where Have You Been
76,Troye Sivan,NaN,"Young ambition,Say we'll go slow but we never ...",pop,Dance to This (feat. Ariana Grande)
77,Anitta,NaN,"When it's all said and done, will I see you?,W...",pop,Will I See You (feat. Poo Bear)


In [7]:
bag_of_words = []
for index, row in df.iterrows():
    new_lyrics = clusterfy(row["lyrics"],kmeans_clusters)
    new_title = clusterfy(row["title"],kmeans_clusters)
    bag_of_words.append(" ".join(new_lyrics + new_title))
    
bag_of_words[0]

'hee hee ooh go on girl aaow hey pretty baby with the high heels on you give me fever like i ve never ever known you re just a product of loveliness i like the groove of your walk your talk your dress i feel your fever from miles around i ll pick you up in my car and we ll paint the town just kiss me baby and tell me twice that you re the one for me the way you make me feel the way you make me feel you really turn me on you really turn me on you knock me off of my feet you knock me off of my feet my lonely days are gone my lonely days are gone i like the feeling you re giving me just hold me baby and i m in ecstasy oh i ll be working from nine to five to buy you things to keep you by my side i never felt so in love before just promise baby you ll love me forever more i swear i m keeping you satisfied cause you re the one for me the way you make me feel the way you make me feel you really turn me on you really turn me on you knock me off of my feet you knock me off of my feet my lonely 

## Running classifiers

In [8]:
vectorizer = CountVectorizer(
    min_df=10,
    lowercase=True,
    strip_accents="unicode",
    stop_words="english",
    ngram_range=(2,2)
)
vectors = vectorizer.fit_transform(bag_of_words)

In [9]:
sss = ShuffleSplit(n_splits=1, test_size=0.3)

In [10]:
classifiers = {
    "Naive Bayes": MultinomialNB(alpha=1),
    "SGCD" : SGDClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNeighborsClassifier" : KNeighborsClassifier(3),
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=5),
}

In [11]:
styles_unique = df["style"].unique()
styles = df["style"]
labels = np.array(styles.tolist())

for train_index, test_index in sss.split(vectors, labels):
    X_train, X_test = vectors[train_index], vectors[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    categories = np.unique(labels[test_index]).tolist()
    
    # instanciamos o classificador
    for classifier_name, classifier in classifiers.items():
        print(f"=========== {classifier_name} =========")
        clf = classifier
        # colocamos os nossos dados de treino
        clf.fit(X_train, y_train)
        # medimos sua acurácia com os dados de teste
        predictions = clf.predict(X_test)
        print(classification_report(y_test, predictions, target_names=categories))
        print(f"SCORE: {clf.score(X_test, y_test)}")
        print('-'*100)

=========== Naive Bayes =========
              precision    recall  f1-score   support

     country       0.38      0.72      0.50       297
       indie       0.33      0.25      0.29       271
         pop       0.42      0.34      0.38       261
      reggae       0.53      0.59      0.56       313
        rock       0.26      0.06      0.10       238

    accuracy                           0.41      1380
   macro avg       0.38      0.39      0.36      1380
weighted avg       0.39      0.41      0.38      1380

SCORE: 0.41231884057971013
----------------------------------------------------------------------------------------------------
=========== SGCD =========
              precision    recall  f1-score   support

     country       0.45      0.39      0.42       297
       indie       0.31      0.35      0.33       271
         pop       0.40      0.34      0.36       261
      reggae       0.44      0.53      0.48       313
        rock       0.23      0.21      0.22       2

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

     country       0.35      0.44      0.39       297
       indie       0.26      0.31      0.28       271
         pop       0.40      0.25      0.30       261
      reggae       0.50      0.57      0.53       313
        rock       0.20      0.13      0.16       238

    accuracy                           0.36      1380
   macro avg       0.34      0.34      0.33      1380
weighted avg       0.35      0.36      0.35      1380

SCORE: 0.35507246376811596
----------------------------------------------------------------------------------------------------
=========== KNeighborsClassifier =========
              precision    recall  f1-score   support

     country       0.28      0.37      0.32       297
       indie       0.22      0.24      0.23       271
         pop       0.45      0.12      0.19       261
      reggae       0.37      0.59      0.46       313
        rock       0.20      0.12      0.15       238

    accuracy  

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## Clustering using agglomerative clusters

In [6]:
with open("agglomerative_clusters.json", "r") as f:
    agglomerative_clusters = json.load(f)

In [7]:
bag_of_words = []
for index, row in df.iterrows():
    new_lyrics = clusterfy(row["lyrics"],agglomerative_clusters)
    new_title = clusterfy(row["title"],agglomerative_clusters)
    bag_of_words.append(" ".join(new_lyrics + new_title))
    
bag_of_words[0]

'amanheceu que pesquei mais ainda era mais um olho cada pensar outro claro fumar normal chifrada desanimado chato a largar ainda e ainda mexicana calma minha Teardrop sabia o que que ainda ainda que normal difícil brazilian vayas ainda que scale marra coisas lágrimas lágrimas e ainda a pesar penso pelo o que ainda te cruzadas e normal cada o ar que é o lado pensar difícil ainda largar a rajada difícil ainda você ainda normal cada o ar lado no pensar da minha ainda eu vou te honrar difícil ainda largar a rajada difícil ainda você ainda normal cada o ar lado no pensar da minha ainda eu vou te honrar jeito eu vou te honrar mexicana calma minha Teardrop sabia o que que ainda ainda que normal difícil brazilian vayas ainda que scale marra coisas lágrimas lágrimas e ainda a pesar penso pelo o que ainda te cruzadas e normal cada o ar que é o lado pensar difícil ainda largar a rajada difícil ainda você ainda normal cada o ar lado no pensar da minha ainda eu vou te honrar difícil ainda largar a 

In [8]:
vectorizer = CountVectorizer(
    min_df=10,
    lowercase=True,
    strip_accents="unicode",
    stop_words="english",
    ngram_range=(2,2)
)
vectors = vectorizer.fit_transform(bag_of_words)

In [9]:
sss = ShuffleSplit(n_splits=1, test_size=0.3)

In [10]:
classifiers = {
    "Naive Bayes": MultinomialNB(alpha=1),
    "SGCD" : SGDClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNeighborsClassifier" : KNeighborsClassifier(3),
    "DecisionTreeClassifier": DecisionTreeClassifier(max_depth=5),
}

In [11]:
styles_unique = df["style"].unique()
styles = df["style"]
labels = np.array(styles.tolist())

for train_index, test_index in sss.split(vectors, labels):
    X_train, X_test = vectors[train_index], vectors[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

    categories = np.unique(labels[test_index]).tolist()
    
    # instanciamos o classificador
    for classifier_name, classifier in classifiers.items():
        print(f"=========== {classifier_name} =========")
        clf = classifier
        # colocamos os nossos dados de treino
        clf.fit(X_train, y_train)
        # medimos sua acurácia com os dados de teste
        predictions = clf.predict(X_test)
        print(classification_report(y_test, predictions, target_names=categories))
        print(f"SCORE: {clf.score(X_test, y_test)}")
        print('-'*100)

=========== Naive Bayes =========
              precision    recall  f1-score   support

 alternativo       0.08      0.02      0.03       308
       blues       0.18      0.05      0.08       285
      bolero       0.25      0.01      0.02       311
       brega       0.18      0.23      0.20       300
     country       0.10      0.37      0.16       290
    cuarteto       0.13      0.30      0.18       295
      cumbia       0.21      0.23      0.22       307
       dance       0.29      0.10      0.15       326
       disco       0.24      0.07      0.11       284
  eletrônica       0.29      0.05      0.09       269
     emocore       0.17      0.09      0.12       310
        fado       0.65      0.40      0.50       309
        folk       0.11      0.06      0.08       313
        funk       0.32      0.65      0.43       291
      grunge       0.11      0.21      0.14       272
    hardcore       0.29      0.01      0.01       314
       house       0.11      0.14      0.13    

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

 alternativo       0.04      0.05      0.05       308
       blues       0.07      0.11      0.08       285
      bolero       0.12      0.20      0.15       311
       brega       0.12      0.28      0.17       300
     country       0.09      0.08      0.09       290
    cuarteto       0.13      0.20      0.16       295
      cumbia       0.14      0.19      0.16       307
       dance       0.17      0.11      0.14       326
       disco       0.16      0.10      0.12       284
  eletrônica       0.12      0.09      0.10       269
     emocore       0.13      0.08      0.10       310
        fado       0.22      0.39      0.28       309
        folk       0.10      0.08      0.09       313
        funk       0.23      0.32      0.26       291
      grunge       0.09      0.20      0.12       272
    hardcore       0.09      0.04      0.05       314
       house       0.11      0.16      0.13       298
       indie       0.08    

/Users/paulocarneiro/MusicClassifier/venv/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
